# Class Tournament

In [ ]:
import random as rand
from tabulate import tabulate
from Match import Match, name, print_history
from Cooperate import Cooperate
from Defect import Defect
from Flipper import Flipper
from TitForTat import TitForTat

cooperate = Cooperate()
defect = Defect()
flipper = Flipper()
titForTat = TitForTat()

In [ ]:
# Import everyone's rules
from AverageTitForTat import AverageTitForTat
from CheckAndChoose import CheckAndChoose, np
from dbTriCon import dbTriCon
from fifty_good import fifty_is_good
from Patrick import Patrick
from scummyPrisoner import scummyPrisoner
from Smart import random, Smart
from suspiciousProbingTFT import random, SPTFT
from TFTMemory import random, TFTMemory
from TitForTwoTats import TitForTwoTats
from Trustworthy import random, Trustworthy
from ResponsiveTFTMod import ResponsiveTFTMod
from suspiciousTitForTat import suspiciousTitForTat

averageTitForTat = AverageTitForTat()
checkAndChoose = CheckAndChoose()
dBTriCon = dbTriCon()
fifty_good = fifty_is_good()
patrick = Patrick()
scummy_Prisoner = scummyPrisoner()
smart = Smart()
susProbingTFT = SPTFT()
tftMemory = TFTMemory()
titForTwoTats = TitForTwoTats()
trustworthy = Trustworthy()
responsiveTFTMod = ResponsiveTFTMod()
susTitForTat = suspiciousTitForTat()

rule_list = [averageTitForTat, checkAndChoose, dBTriCon, fifty_good, patrick, scummy_Prisoner, smart, susProbingTFT, tftMemory, titForTwoTats, trustworthy, cooperate, defect, flipper, titForTat, responsiveTFTMod, susTitForTat]

In [ ]:
class Tournament():

    def __init__(self, rule_list, length, identical_match=False):
        self.rule_list = rule_list
        self.length = length
        self.identical_match = identical_match
        
        self.match_scores = []
        self.rule_scores = {}
        self.win_diff = []
        self.wins = []
        
        for rule in self.rule_list:
            self.rule_scores[name(rule)] = []

    def run(self):
        for match in self.match_gen():
            match.run()
            score = match.score()
#             self.rule_scores[name(match.rule0)][name(match.rule1)] = score[match.rule0.order]
#             self.rule_scores[name(match.rule1)][name(match.rule0)] = score[match.rule1.order]
#             self.match_scores[match.name] = score
            self.match_scores.append((match.name, score))
    
            self.rule_scores[name(match.rule0)].append(score)
            self.rule_scores[name(match.rule1)].append(score[::-1])

    def match_gen(self):
        ''' Generator function to return initialized matches from the list of rules'''
        l = range(len(self.rule_list))
        if self.identical_match:
            n = 0
        else:
            n = 1
        for i in l:
            for j in l:
                
                if j < i+n:
                    continue
                else:
                    new_match = Match(self.rule_list[i], self.rule_list[j], self.length)
                    yield new_match
                    
    def all_scores(self):
        
        self.score_loss_wins()
        pass
        
    
    def sort_rule_score(self):
        
        [scores.sort(reverse=True) for key, scores in self.match_scores.items()]
    
    def score_loss_wins(self):
        ''' Perform scorings based on number of wins/losses/draws. '''
        for rule, scores in self.rule_scores.items():
            new_scores = [(score[0]-score[1]) for score in scores]
            self.win_diff.append((rule, new_scores))
            
            win = sum([1 if (score>0) else 0 for score in new_scores])
            draw = sum([1 if (score==0) else 0 for score in new_scores])
            loss = sum([1 if (score<0) else 0 for score in new_scores])
            self.wins.append((rule, win, draw, loss))    

In [ ]:
t = Tournament(rule_list, 100)
t.run()
t.all_scores()

## Wins, Losses, and Draws

In [ ]:
s_w = sorted(t.wins, key=lambda x: x[1], reverse=True)
s_d = sorted(t.wins, key=lambda x: x[2], reverse=True)
s_l = sorted(t.wins, key=lambda x: x[3], reverse=True)

headers = ['Wins', 'Draws', 'Losses']

print(tabulate([[s_w[i], s_d[i], s_l[i]] for i in range(len(s_w))], headers=headers))

## Raw Points

#### Maximum Points in a Match

In [ ]:
s_ind = sorted(t.match_scores, key=lambda x: max(x[1]), reverse=True)
s_col = sorted(t.match_scores, key=lambda x: sum(x[1]), reverse=True)

headers = ['Individual', 'Collective']

tab_list = []
for i in range(len(s_ind)):
    split = s_ind[i][0].split('-')
    if s_ind[i][1][0] > s_ind[i][1][1]:
        rule = split[0]
    else:
        rule = split[1]
    
    indiv = (rule, s_ind[i][1])
    tab_list.append([indiv, s_col[i]])
    # tab_list.append([s_ind[i], s_col[i]])

print(tabulate(tab_list, headers=headers))

#### Maximum Total Points

In [ ]:
max_list = []

for rule, scores in t.rule_scores.items():
    total = str(sum([score[0] for score in scores]))
    avg = str(sum([score[0] for score in scores])/len(scores))
    max_list.append([rule, total, avg])

max_list.sort(key=lambda x: x[1], reverse=True)
print(tabulate(max_list, headers=['Rule', 'Total Points', 'Average Points']))

## Point Differences
Which rule won by the most points on average? 
Which lost by the least on average?

In [ ]:
t.rule_scores

gr_diff_list = []
le_diff_list = []
l = len(t.rule_scores)
for rule, scores in t.rule_scores.items():
    grs = 0
    les = 0
    for score in scores:
        if score[0] > score[1]:
            grs += score[0] - score[1] 
        elif score[1] > score[0]:
            les += score[1] - score[0]
    gr_diff_list.append([rule, str(grs/l)])
    le_diff_list.append([rule, str(les/l)])
        
gr_diff_list.sort(key=lambda x: float(x[1]), reverse=True)
le_diff_list.sort(key=lambda x: float(x[1]))

diff_list = []
for i in range(len(gr_diff_list)):
    diff_list.append([gr_diff_list[i], le_diff_list[i]])

headers = ['Won by Most', 'Lost by Least']

print(tabulate(diff_list, headers=headers))